<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/IonQ_%EC%96%91%EC%9E%90%ED%9A%8C%EB%A1%9C_2_%EC%A0%9C%EC%96%B4%EC%97%B0%EC%82%B0_Gate%EB%A7%8C%EB%93%9C%EB%8A%94_%EB%B0%A9%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install qiskit[visualization]
!python -m pip install qiskit-aer
!python -m pip install qiskit-ionq

In [ ]:
#import IonQ backends
from qiskit_ionq import IonQProvider
ionq_provider = IonQProvider(token='sIdNYBF7UGuQj2LDYeYQbR2wpaxJJU4K')

#Import other qiskit requirements
from qiskit import QuantumCircuit, Aer, execute
from qiskit. tools.visualization import plot_histogram, array_to_latex #Vector Martix를 좀더 fancy하게 표현하기 위함
from qiskit.extensions import UnitaryGate #사용자 정의 양자게이트를 만든는 용도


**Qustion 1: Implementing CNOT (Simulator vs. Hardware)**


In [ ]:
qc = QuantumCircuit(2, 2) #Instantiate a circuit with two qubits and two classical bits
qc.cx(0, 1) #Apply the CNOT gate with control qubit 0 and target qubit 1

# Measure all the qubits
for i in range(len(qc.qubits)):
  qc.measure(i, i)
###for

#Draw the circuit to visualize it
qc.draw()

In [ ]:
backend = Aer.get_backend('qasm_simulator') #choose your backend
job = execute(qc, backend) #create the job object
result = job.result() #create the result object
counts = result.get_counts() #get the counts dictionary
plot_histogram(counts) #plot the histogram of counts

In [ ]:
backend = ionq_provider.get_backend('ionq_qpu') #choose your backend
job = execute(qc, backend) #create the job object


# Check if job is complete
from qiskit.providers.jobstatus import JobStatus
import time
while job.status() is not JobStatus.DONE:
  print("Job Status is ", job.status())
  time.sleep(5)
###while

result = job.result() #create the result object
counts = result.get_counts() #get the counts dictionary
plot_histogram(counts) #plot the histogram of counts

In [ ]:
# First we will create our very own, new quantum gate

my_circuit = QuantumCircuit(2) # Our gate is a two-qubit gate, so initialize a two-qubit quantum circuit
my_circuit.x(0) # our gate has a NOT gate
my_circuit.cx(0, 1)# our gate then has a CNOT gate
my_circuit.x(0) # then we have a NOT Gate
my_circuit.draw() #Visualization

In [ ]:
#convert your circuit to a gate
xcx = my_circuit.to_gate(label="XCX")
type(xcx)

In [ ]:
qc = QuantumCircuit(2, 2) #Instantiate a circuit with two qubits and two clasical bits
qc.append(xcx, [0, 1])

# Measure all the qubits
for i in range(len(qc.qubits)):
  qc.measure(i, i)
###for

qc.draw()

In [ ]:
backend = ionq_provider.get_backend('ionq_simulator') #choose your backend
job = execute(qc, backend) #create the job object


# Check if job is complete
from qiskit.providers.jobstatus import JobStatus
import time
while job.status() is not JobStatus.DONE:
  print("Job Status is ", job.status())
  time.sleep(5)
###while

result = job.result() #create the result object
counts = result.get_counts() #get the counts
plot_histogram(counts)

In [ ]:
#First we will create the regular V-gate
matrix = [[0.5 * (1 - 1j), 0.5 * (1 - 1j) * 1j],
          [0.5 * (1 - 1j) * 1j, 0.5 * (1 - 1j)]] #enter the marix the V-gate
v = UnitaryGate(matrix, 'V') #Defining V
#Trying out our V-gate
qc = QuantumCircuit(1) #Initialise Quantum Circuit
qc.unitary(v, 0) #Act unitary on circuit
print(qc.draw()) #Visualize circuit
#matrix of V-gate
backend = Aer.get_backend('unitary_simulator') #use unitary_simulator as backend
job = execute(qc, backend) #create job object
result = job.result() #create result object
matrix = result.get_unitary() #get the unitary martix corresponding to the circuit
array_to_latex(matrix)#illustrate the matrix

In [ ]:
controlled_v = v.control() #define the controlled-v operation
# Trying out the controlled-v gate
qc = QuantumCircuit(2) #Define a Quantum Circuit
qc.append(controlled_v, [0, 1]) #Append the controlled_v gate
qc.draw() #Visualize

In [ ]:
v_dagger = v.adjoint() #define v-dagger
controlled_v_dagger = v_dagger.control() #define the controlled-v-dagger

#Trying it out
qc = QuantumCircuit(1) #Define a quantum circuit
qc.unitary(v_dagger, 0) #implement v-dagger
print(qc.draw()) #implement v-dagger

#matrix of V-dagger
backend = Aer.get_backend('unitary_simulator') #use unitary_simulator as backend
job = execute(qc, backend) #create job object
result = job.result() #create result object
matrix = result.get_unitary() #get the unitary martix corresponding to the circuit
array_to_latex(matrix)#illustrate the matrix


In [ ]:
qc = QuantumCircuit(3) # A Toffoli gate is a 3-qubit gate. So define a 3-qubit quantum circuit
qc.append(controlled_v, [1, 2]) # Append your controlled-V operation
qc.cx(0, 1) # CNOT
qc.append(controlled_v_dagger, [1, 2]) # Controlled-V dagger
qc.cx(0, 1) # CNOT
qc.append(controlled_v, [0, 2]) # One more controole-V
print(qc.draw()) # Visualize

# Let us check to see if we got the right transformation. Let's get the matrix
#matrix of V-dagger
backend = Aer.get_backend('unitary_simulator') #use unitary_simulator as backend
job = execute(qc, backend) #create job object
result = job.result() #create result object
matrix = result.get_unitary() #get the unitary martix corresponding to the circuit
array_to_latex(matrix)#illustrate the matrix

In [ ]:
# Let's compare with the martrix of the Toffoli gate
qc = QuantumCircuit(3) # Create 3-qubit quantum circuit
qc.ccx(0, 1, 2) # apply Toffoli gate
print(qc.draw()) # Visualize

#matrix of V-dagger
backend = Aer.get_backend('unitary_simulator') #use unitary_simulator as backend
job = execute(qc, backend) #create job object
result = job.result() #create result object
matrix = result.get_unitary() #get the unitary martix corresponding to the circuit
array_to_latex(matrix)#illustrate the matrix

In [ ]:
qc.decompose().draw()

In [ ]:
qc.decompose().decompose().draw()

In [ ]:
qc.decompose().decompose().decompose().draw()

In [ ]:
qc.decompose().decompose().decompose().decompose().draw()

In [ ]:
#End of Codes